安装

In [ ]:
pip install lightgbm

导入模块

In [ ]:
import lightgbm as lgb

敲定好一组参数

In [ ]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'auc'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

调参 http://www.huaxiaozhuan.com/%E5%B7%A5%E5%85%B7/lightgbm/chapters/lightgbm_usage.html

| 树的参数 | 含义 | 设置 |
|--|--|--|
| num_leaves | 叶节点的数目，控制树模型复杂度 | 小于2^{depth} |
| min_data_in_leaf | 每个叶节点的最少样本数量 | 控制过拟合 |
| max_depth | 树的最大深度 | - |

| 核心参数 | 含义 | 设置 |
|--|--|--|
| task | 要执行的任务 | train/predict/convert_model |
| application 或者objective 或者 app | 任务类型 | regression/binary/multiclass/xentropy/lambdarank/... |
| boosting或者boost或者boosting_type | 基学习器模型算法 | gbdt/rf/dart/goss |
| num_iteration或者num_tree或者 num_round或者 num_boost_round | 迭代次数 | 默认100 |
| learning_rate | 学习率 | 默认为 0.1 |
|num_leaves或者num_leaf | 一棵树上的叶子数 | 默认为 31 |

| 学习控制参数 | 含义 | 设置 |
|--|--|--|
| max_depth | 树模型的最大深度 | 默认值为-1 |
| min_data_in_leaf | 一个叶子节点上包含的最少样本数量。 | 默认值为 20 |
| feature_fraction | 如0.8 表示：在每棵树训练之前选择80% 的特征来训练 | 取值范围为[0.0,1.0]， 默认值为1.0 |
| bagging_fraction 或者 subsample  | 如0.8 表示：在每棵树训练之前选择80% 的样本（非重复采样）来训练 | 取值范围为[0.0,1.0]， 默认值为1.0 |
| early_stopping_round或者early_stopping  | 如果一个验证集的度量在early_stopping_round 循环中没有提升，则停止训练 | - |
| lambda_l1 或者reg_alpha | 表示L1正则化系数。 | 默认为0 |
| lambda_l2 或者reg_lambda | 表示L2正则化系数。 | 默认为0 |
| min_split_gain 或者min_gain_to_split | 一个浮点数，表示执行切分的最小增益 | 默认为0 |
| min_data_per_group | 表示每个分类组的最小数据量 用于排序任务 | 默认值为100 |
| cat_smooth | 用于category 特征的概率平滑，降低噪声在category 特征中的影响，尤其是对于数据很少的类。 | 默认值为 10 |

| 度量参数 | 含义 | 设置 |
|--|--|--|
| metric | 度量的指标 | 对于回归问题，使用l2 ； 对于二分类问题，使用binary_logloss；对于lambdarank 问题，使用ndcg |
| metric_freq或者'output_freq | 一个正式，表示每隔多少次输出一次度量结果 | 默认为1 |
| train_metric 或者training_metric | 如果为True，则在训练时就输出度量结果 | 默认值为 False |
| ndcg_at 或者 ndcg_eval_at 或者eval_at | 指定了NDCG 评估点的位置。 | 默认为1,2,3,4,5 |

获取更好的准确率：

- 使用较大的 max_bin （学习速度可能变慢）
- 使用较小的 learning_rate 和较大的 num_iterations
- 使用较大的 num_leaves （可能导致过拟合）
- 使用更大的训练数据
- 尝试 dart
- 缓解过拟合：

使用较小的 max_bin

- 使用较小的 num_leaves
- 使用 min_data_in_leaf 和 min_sum_hessian_in_leaf
- 通过设置 bagging_fraction 和 bagging_freq 来使用 bagging
- 通过设置 feature_fraction 来使用特征子抽样
- 使用更大的训练数据
- 使用 lambda_l1, lambda_l2 和 min_gain_to_split 来使用正则
- 尝试 max_depth 来避免生成过深的树

分类特征支持

要想使用categorical 特征，则启用categorical_feature 参数
- 首先要将categorical 特征的取值转换为非负整数，而且如果是连续的范围更好
- 然后使用min_data_per_group 和 cat_smooth 去处理过拟合（当样本数较小，或者category 取值范围较大时）

训练

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

保存模型

In [ ]:
gbm.save_model('model.m')

模型的载入与预测

In [ ]:
gbm = lgb.Booster(model_file='./model/model.m')
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

评估

In [ ]:
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test, y_pred) ** 0.5)

自定义损失函数

In [ ]:
# 类似在xgboost中的形式
# 自定义损失函数需要
def loglikelood(preds, train_data):
    labels = train_data.get_label()
    preds = 1. / (1. + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1. - preds)
    return grad, hess


# 自定义评估函数
def binary_error(preds, train_data):
    labels = train_data.get_label()
    return 'error', np.mean(labels != (preds > 0.5)), False


gbm = lgb.train(params,
                lgb_train,
                num_boost_round=10,
                init_model=gbm,
                fobj=loglikelood,
                feval=binary_error,
                valid_sets=lgb_eval)

print('用自定义的损失函数与评估标准完成第40-50轮...')

optuna超参数优化

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_uniform('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

study.best_params  # E.g. {'x': 2.002108042}

具体使用可参考《GIVE OPTUNA A SHOT!》
https://zhuanlan.zhihu.com/p/138521995